In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("GOOGLE_GENAI_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["USER_AGENT"] = "BasicRAG/1.0"  

Initializing LLM

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192", api_key=groq_api_key)

Initializing Embedding

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=api_key
    )

Loading Files

In [4]:
from langchain_community.document_loaders import WebBaseLoader

async def load_webpage(url):
    loader = WebBaseLoader(web_paths=[url])
    docs = [doc async for doc in loader.alazy_load()]
    return docs

In [5]:
from langchain_community.document_loaders import PyPDFLoader

async def load_pdf(file_path):
    loader = PyPDFLoader(file_path)
    pages = [page async for page in loader.alazy_load()]
    return pages

In [6]:
async def dispatcher(input_data):
    if input_data.lower().startswith("http"):
        return await load_webpage(input_data)
    elif input_data.lower().endswith(".pdf"):
        return await load_pdf(input_data)
    else:
        raise ValueError("Unsupported file type or source!")
    
async def process_input(input_data):
    try:
        result = await dispatcher(input_data)
        return result
    except Exception as e:
        return str(e)

Initializing Text Splitter

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=130,
    add_start_index=True
)

Initializing vector store

In [8]:
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore

dimension = 768
index = faiss.IndexFlatL2(dimension)
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
    )

Text Splitting, Storing in vector_store, Retrieval and Generation

In [9]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

async def rag_chain_fn(messages, history, file=None):
    if file:
        docs = await process_input(file)  
        all_splits = text_splitter.split_documents(docs)
        vector_store.add_documents(documents=all_splits)
    retriever = vector_store.as_retriever()

    contextualize_q_system_prompt = (
        "Given a chat history and the latest user question "
        "which might reference context in the chat history, "
        "formulate a standalone question which can be understood "
        "without the chat history. Do NOT answer the question, "
        "just reformulate it if needed and otherwise return it as is."
    )

    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )

    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    ) 

    system_prompt = (
        """
        You are a knowledgeable and helpful assistant capable of answering questions accurately and concisely.
        Use the provided context below to guide your responses when relevant:
        {context}
        For general questions or when the context is not directly applicable, respond naturally and confidently based on your broader knowledge.
        If you are unsure or lack the necessary information, acknowledge it politely.
        Avoid explicitly stating whether your response is based on the provided context or your broader knowledge unless asked directly.
        """
    )

    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}")
        ]
    )

    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

    # Invoke the chain with input and history
    return rag_chain.invoke({"input": messages, "chat_history": history})["answer"]


In [ ]:
import gradio as gr

app = gr.ChatInterface(
    fn=rag_chain_fn,
    type="messages",
    additional_inputs=[gr.Textbox(label="Enter a file path that ends with '.pdf' or a url that begins with 'http'.")]
)

app.launch()